In [1]:
import os
import subprocess
import sys
sys.path.append('..')
from tools.formating.formating import *
from tools.utils.datasplit import *
from tools.qc.scanpy_qc import *
import numpy as np
import scipy

R[write to console]: Loading required package: SingleCellExperiment

R[write to console]: Loading required package: SummarizedExperiment

R[write to console]: Loading required package: MatrixGenerics

R[write to console]: Loading required package: matrixStats

R[write to console]: 
Attaching package: ‘MatrixGenerics’


R[write to console]: The following objects are masked from ‘package:matrixStats’:

    colAlls, colAnyNAs, colAnys, colAvgsPerRowSet, colCollapse,
    colCounts, colCummaxs, colCummins, colCumprods, colCumsums,
    colDiffs, colIQRDiffs, colIQRs, colLogSumExps, colMadDiffs,
    colMads, colMaxs, colMeans2, colMedians, colMins, colOrderStats,
    colProds, colQuantiles, colRanges, colRanks, colSdDiffs, colSds,
    colSums2, colTabulates, colVarDiffs, colVars, colWeightedMads,
    colWeightedMeans, colWeightedMedians, colWeightedSds,
    colWeightedVars, rowAlls, rowAnyNAs, rowAnys, rowAvgsPerColSet,
    rowCollapse, rowCounts, rowCummaxs, rowCummins, rowCumprods,
    rowC


    an issue that caused a segfault when used with rpy2:
    https://github.com/rstudio/reticulate/pull/1188
    Make sure that you use a version of that package that includes
    the fix.
    

R[write to console]: Attaching SeuratObject

R[write to console]: Seurat v4 was just loaded with SeuratObject v5; disabling v5 assays and
validation routines, and ensuring assays work in strict v3/v4
compatibility mode

R[write to console]: 
Attaching package: ‘Seurat’


R[write to console]: The following object is masked from ‘package:SummarizedExperiment’:

    Assays


R[write to console]: Registered S3 method overwritten by 'SeuratDisk':
  method            from  
  as.sparse.H5Group Seurat

R[write to console]: Signac built for for SeuratObject v4 was just loaded with SeuratObject
v5; disabling v5 assays and validation routines, and ensuring assays
work in strict v3/v4 compatibility mode



scanpy==1.9.5 anndata==0.9.2 umap==0.5.4 numpy==1.22.4 scipy==1.9.1 pandas==1.3.5 scikit-learn==1.3.1 statsmodels==0.14.0 igraph==0.10.8 louvain==0.8.1 pynndescent==0.5.10


In [2]:
# input = "/ps/ai-ready/data/Clustering/Tabula_Sapiens/TS_germ line.h5ad"
# input = "/ps/ai-ready/data/Clustering/Tabula_Muris/droplet_Bladder_seurat_tiss.h5ad"
input = "/ps/ai-ready/data/error/droplet_Bladder_seurat_tiss_qc_result.h5ad"
# input = "/ps/ai-ready/data/Clustering/Tabula_Sapiens/TS_Skin.h5ad"
# input = "/ps/ai-ready/data/error/droplet_Bladder_seurat_tiss.h5seurat"
# input = "/ps/ai-ready/data/error/tung.rds"
adata = load_anndata(input)
adata

/ps/ai-ready/data/error/droplet_Bladder_seurat_tiss_qc_result.h5ad


AnnData object with n_obs × n_vars = 2500 × 23341
    obs: 'orig.ident', 'n_counts', 'n_genes', 'channel', 'tissue', 'subtissue', 'mouse.sex', 'mouse.id', 'percent.ercc', 'percent.ribo', 'free_annotation', 'cell_ontology_class', 'res.0.4', 'previous_free_annotation', 'previous_cell_ontology_class', 'cluster.ids', 'cell_ontology_id', 'pct_counts_mt', 'pct_counts_rb', 'pct_counts_hb', 'percent.plat', 'RNA_snn_res.1', 'seurat_clusters', 'doublet_score', 'doublet_class'
    var: 'vst.mean', 'vst.variance', 'vst.variance.expected', 'vst.variance.standardized', 'vst.variable'
    uns: 'neighbors'
    obsm: 'X_pca', 'X_tsne', 'X_umap', 'X_umap_3D'
    layers: 'scale.data'
    obsp: 'connectivities', 'distances'

In [4]:
adata.obsm['X_umap_3D'].shape

(2500, 3)

In [5]:
adata.obsm['X_umap'].shape

(2500, 2)

In [4]:
adata.obs

,NRP,age_group,cell_source,cell_type,donor,gender,n_counts,n_genes,percent_mito,percent_ribo,region,sample,scrublet_score,source,type,version,cell_states,Used
AACCCAAAGTACGAGC-1-H0015_apex,No,50-55,Harvard-Nuclei,Neuronal,H5,Female,5796.0,3033,0.004141,0.001898,AX,H0015_apex,0.139344,Nuclei,DBD,V3,NC3,Yes
AACCTGATCAGCTTCC-1-H0015_apex,No,50-55,Harvard-Nuclei,Neuronal,H5,Female,3295.0,1696,0.000607,0.001517,AX,H0015_apex,0.027304,Nuclei,DBD,V3,NC4,Yes
AACGAAACACGACTAT-1-H0015_apex,No,50-55,Harvard-Nuclei,Neuronal,H5,Female,743.0,567,0.008075,0.001346,AX,H0015_apex,0.015206,Nuclei,DBD,V3,NC1,Yes
AAGTGAATCGAGAGCA-1-H0015_apex,No,50-55,Harvard-Nuclei,Neuronal,H5,Female,562.0,441,0.005338,0.005338,AX,H0015_apex,0.014286,Nuclei,DBD,V3,NC1,Yes
AAGTGAATCTGATTCT-1-H0015_apex,No,50-55,Harvard-Nuclei,Neuronal,H5,Female,984.0,723,0.004065,0.000000,AX,H0015_apex,0.015206,Nuclei,DBD,V3,NC1,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
CAACAACTCCGAACGC-1-HCAHeart8102861,Yes,60-65,Sanger-CD45,Neuronal,D11,Female,2457.0,1271,0.157102,0.028490,SP,HCAHeart8102861,0.139344,CD45+,DCD,V3,NC1,Yes
TCAGCAACAGCTACCG-1-HCAHeart8102861,Yes,60-65,Sanger-CD45,Neuronal,D11,Female,3995.0,1797,0.158698,0.034793,SP,HCAHeart8102861,0.125475,CD45+,DCD,V3,NC3,Yes
CACACAAGTCTCAGGC-1-HCAHeart8102862,Yes,60-65,Sanger-CD45,Neuronal,D11,Female,1563.0,902,0.145873,0.031350,AX,HCAHeart8102862,0.098229,CD45+,DCD,V3,NC1,Yes
GAGAAATTCTAGTACG-1-HCAHeart8102862,Yes,60-65,Sanger-CD45,Neuronal,D11,Female,1995.0,1032,0.193985,0.030075,AX,HCAHeart8102862,0.108062,CD45+,DCD,V3,NC3,Yes


In [7]:
adata.obsm['X_umap'].shape

(3961, 2)

In [6]:
df = pd.DataFrame.sparse.from_spmatrix(adata.X, index=None, columns=None)
df

,0,1,2,3,4,5,6,7,8,9,...,23331,23332,23333,23334,23335,23336,23337,23338,23339,23340
0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,2.0,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,1.0,1.0,0.0,2.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,2.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2495,0.0,0.0,0.0,2.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2496,0.0,0.0,0.0,0.0,0.0,2.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2497,0.0,0.0,0.0,0.0,0.0,3.0,0.0,2.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2498,0.0,0.0,0.0,4.0,0.0,2.0,0.0,4.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0


In [ ]:
# Convert Seurat/Single-Cell Experiment object to Anndata object and return the path of Anndata object
def convert_seurat_sce_to_anndata1(path, assay='RNA'):
    import rpy2.rinterface_lib.callbacks as rcb
    import rpy2.robjects as ro
    import anndata2ri
    from rpy2.robjects.packages import importr
    from rpy2.robjects import pandas2ri
    from rpy2.robjects.conversion import localconverter
    
    rcb.logger.setLevel(logging.ERROR)
    ro.pandas2ri.activate()
    anndata2ri.activate()

    # Defining the R script and loading the instance in Python
    ro.r['source'](os.path.abspath("../tools/formating/formating.R"))
    # Access the loaded R functions
    ConvertSeuratSCEtoAnndata_r = ro.globalenv['ConvertSeuratSCEtoAnndata']

    assay_names = None
    adata_path = None
    results = None

    if path.endswith(".h5Seurat") or path.endswith(".h5seurat") or path.endswith(".rds") or path.endswith(".Robj"):
        results = ConvertSeuratSCEtoAnndata_r(path, assay=assay)
        print(results)
        adata_path = str(results.rx2('anndata_path'))
        # adata_path = list(results[2])[0]
        assay_names = list(results[1])
    
    return results

In [ ]:
results = convert_seurat_sce_to_anndata1(input)
results

In [ ]:
type(results)

In [ ]:
from collections import OrderedDict

results = OrderedDict(results)

In [ ]:
results['anndata_path'][0]